### NPSC psudeo-extension stands for NuPlan Scenario Collection. 
#### It is a pkl file with the naming convention `filename.npsc.pkl` that contains:
* The data structure of a dictionary of scenario modifications.
* The `DATASET_PARAMS` list of filters in order to extract the correct scenarios to modify.

In [24]:
import logging
import random
import nest_asyncio
import pandas as pd

nest_asyncio.apply()

import torch

torch.cuda.is_available()

import os
from pathlib import Path
import tempfile

from shutil import rmtree
from typing import List, Optional, Union

from nuplan.planning.script.run_simulation import main as main_simulation

### If your paths arent set correctly in the environment, set them here manually.

In [ ]:
%env NUPLAN_DATA_ROOT=../../../../datasets/nuplan/dataset
%env NUPLAN_MAPS_ROOT=../../data/nuplan/maps
%env NUPLAN_EXP_ROOT=../../data/nuplan/exp
%env NUPLAN_DEVKIT_ROOT=../../nuplan-devkit/
%pwd

In [26]:
import hydra
import pytorch_lightning as pl
from omegaconf import DictConfig, OmegaConf

from nuplan.common.utils.s3_utils import is_s3_path
from nuplan.planning.script.builders.simulation_builder import build_simulations
from nuplan.planning.script.builders.simulation_callback_builder import (
    build_callbacks_worker,
    build_simulation_callbacks,
)
from nuplan.planning.script.utils import (
    run_runners,
    set_default_path,
    set_up_common_builder,
)
from nuplan.planning.simulation.planner.abstract_planner import AbstractPlanner

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# If set, use the env. variable to overwrite the default dataset and experiment paths
set_default_path()

# If set, use the env. variable to overwrite the Hydra config
CONFIG_PATH = os.getenv("NUPLAN_HYDRA_CONFIG_PATH", "config/simulation")

if os.environ.get("NUPLAN_HYDRA_CONFIG_PATH") is not None:
    CONFIG_PATH = os.path.join("../../../../", CONFIG_PATH)

if os.path.basename(CONFIG_PATH) != "simulation":
    CONFIG_PATH = os.path.join(CONFIG_PATH, "simulation")
CONFIG_NAME = "default_simulation"

In [27]:
from tutorials.utils.tutorial_utils import construct_simulation_hydra_paths

# Location of paths with all simulation configs
BASE_CONFIG_PATH = os.path.join(
    os.getenv("NUPLAN_TUTORIAL_PATH", ""), "../nuplan/planning/script"
)
simulation_hydra_paths = construct_simulation_hydra_paths(BASE_CONFIG_PATH)

In [ ]:
import pickle
#pdm_closed_pdm_closed_cross_pure_scen_20
#pdm_closed_pdm_closed_seq_pure_scen_100
#pdm_closed_pdm_closed_merge_pure_scen_20
#pdm_closed_pdm_closed_diverge_pure_scen_1400
EXPERIMENT_NAME = "choose_saved_experiment_name"
modification_file_path = "npsc_files/" + EXPERIMENT_NAME + ".npsc.pkl"
with open(modification_file_path, "rb") as output_file:
    mod_list_dict = pickle.load(output_file)
    print(list(mod_list_dict.keys()))
    DATASET_PARAMS = pickle.load(output_file)

print(DATASET_PARAMS)

In [29]:
urban_ckpt = "/home/ehdykhne/nuplan-devkit/experiments/pretrained_checkpoints/urbandriver_checkpoint.ckpt"
gc_pgp_ckpt = "/home/ehdykhne/nuplan-devkit/experiments/pretrained_checkpoints/gc_pgp_checkpoint.ckpt"
hybrid_ckpt = "/home/ehdykhne/nuplan-devkit/experiments/pretrained_checkpoints/pdm_offset_checkpoint.ckpt"

### Prep first regime. Select appropriate planners and observations from comments below.

In [ ]:
# Initialize configuration management system
hydra.core.global_hydra.GlobalHydra.instance().clear()  # reinitialize hydra if already initialized
hydra.initialize(config_path=simulation_hydra_paths.config_path)

# Compose the configuration
print(simulation_hydra_paths.config_name)
cfg_occ = hydra.compose(
    config_name=simulation_hydra_paths.config_name,
    overrides=[
        # planner
        ################################################
        "planner=pdm_closed_planner",  # [ml_planner, pdm_hybrid_planner, pdm_closed_planner, idm_planner, log_future_planner, simple_planner]
        # planner occlusion
        ################################################
        "+occlusion_cfg.occlusion=true",
        "+occlusion_cfg.manager_type=wedge",  # options: [range, shadow, wedge]
        "+occlusion_cfg.uncloak_reaction_time=1.5",
        "+occlusion_cfg.notice_threshold=1.0",
        ################################################
        # obs
        ################################################
        f"observation.planner_type=pdm_closed",
        ################################################
        # obs occlusion
        ################################################
        f"observation.occlusion_cfg.occlusion=True",
        f"observation.occlusion_cfg.manager_type=wedge",
        f"+observation.occlusion_cfg.uncloak_reaction_time=1.5",
        f"+observation.occlusion_cfg.notice_threshold=1.0",
        ################################################
        # scenario modification
        "+modify_scenario_simulations=true",
        "+second_testing_round=true",
        f"+modification_file_path={modification_file_path}",
        # "+remove_other_agents=true",
        ################################################
        # misc
        ################################################
        "+simulation=closed_loop_multiagent",  # [open_loop_boxes, closed_loop_nonreactive_agents, closed_loop_reactive_agents, closed_loop_multiagent]
        "worker=ray_distributed",  # [sequential, ray_distributed]
        "hydra.searchpath=[pkg://tuplan_garage.planning.script.config.common, pkg://tuplan_garage.planning.script.config.simulation, pkg://nuplan.planning.script.config.common, pkg://nuplan.planning.script.experiments]",
        *DATASET_PARAMS,
        ################################################
    ],
)

output_folder_occ = cfg_occ.output_dir
print('output_folder_occ = "' + output_folder_occ + '"')

## Below are config options to run other types of planners and observations (observations being non-ego vehicles):

# planner: pdm_closed
##########################################
# "planner=pdm_closed_planner",  # [ml_planner, pdm_hybrid_planner, pdm_closed_planner, idm_planner, log_future_planner, simple_planner]
##########################################

# obs: pdm_closed
##########################################
# f"observation.planner_type=pdm_closed",
##########################################

# planner: urban driver
##########################################
# "planner=ml_planner",  # [ml_planner, pdm_hybrid_planner, pdm_closed_planner, idm_planner, log_future_planner, simple_planner]
# "model=urban_driver_open_loop_model",
# "planner.ml_planner.model_config=${model}",
# f"planner.ml_planner.checkpoint_path={urban_ckpt}",
##########################################

# obs: urban driver
##########################################
# "+observation.model=urban_driver_open_loop_model",
# f"observation.planner_type=ml",
# "+observation.model_config=${model}",
# f"observation.checkpoint_path={urban_ckpt}",
##########################################

# planner: gc_pgp
################################################
# "planner=ml_planner",  # [ml_planner, pdm_hybrid_planner, pdm_closed_planner, idm_planner, log_future_planner, simple_planner]
# "model=gc_pgp_model",
# "model.aggregator.pre_train=false",
# "planner.ml_planner.model_config=${model}",
# f"planner.ml_planner.checkpoint_path={gc_pgp_ckpt}",
###############################################

# obs: gc_pgp
###############################################
# "+observation.model_config=${model}",
# "+observation.model=gc_pgp_model",
# "+observation.model.aggregator.pre_train=false",
# f"observation.planner_type=ml",
# f"observation.checkpoint_path={gc_pgp_ckpt}",
################################################

### Now, we run the simulations

In [ ]:
# Run the simulation loop (real-time visualization not yet supported, see next section for visualization)
main_simulation(cfg_occ)

## Prep NuBoard

In [ ]:
import re

output_folder = [output_folder_occ]

print(output_folder_occ)

CONFIG_PATH = "../nuplan/planning/script/config/nuboard"
CONFIG_NAME = "default_nuboard"

# Initialize configuration management system
hydra.core.global_hydra.GlobalHydra.instance().clear()  # reinitialize hydra if already initialized
hydra.initialize(config_path=CONFIG_PATH)
scenario_builder = None
for item in DATASET_PARAMS:
    parts = re.split("=", item, 1)
    if parts[0] == "scenario_builder":
        scenario_builder = parts[1]
if scenario_builder is None:
    raise ValueError("scenario_builder not specified in the dataset parameters")
print(scenario_builder)
# Compose the configuration
cfg = hydra.compose(
    config_name=CONFIG_NAME,
    overrides=[
        f"scenario_builder={scenario_builder}",  # set the database (same as simulation) used to fetch data for visualization
        f"simulation_path={output_folder}",  # [output_folder, output_folder_alt] nuboard file path(s), if left empty the user can open the file inside nuBoard
    ],
)

## Launch NuBoard

In [ ]:
from nuplan.planning.script.run_nuboard import main as main_nuboard

# Run nuBoard
main_nuboard(cfg)

## Below, some visualizations are avaliable if you care to run them. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from vis_utils import circular_hist

%matplotlib inline

# # seq
# output_folder_occ = (
#     "../../data/nuplan/exp/exp/simulation/closed_loop_multiagent/change_to_seq_run"
# )
# # cross
# output_folder_occ = (
#     "../../data/nuplan/exp/exp/simulation/closed_loop_multiagent/change_to_cross_run"
# )
# # merge
# output_folder_occ = (
#     "../../data/nuplan/exp/exp/simulation/closed_loop_multiagent/change_to_merge_run"
# )
# # diverge
# output_folder_occ = (
#     "../../data/nuplan/exp/exp/simulation/closed_loop_multiagent/change_to_diverge_run"
# )

file_path = output_folder_occ + "/metrics/" + "no_ego_at_fault_collisions.parquet"
df = pd.read_parquet(file_path)
relative_angles_of_impact_df = df[
    "relative_impact_angle_at_first_ego_collision_with_vehicles_stat_value"
]
relative_angles_of_impact = relative_angles_of_impact_df.to_numpy()

# fig, ax = plt.subplots(1, 2)
# fig.tight_layout()
figp, axp = plt.subplots(1, 2, subplot_kw=dict(projection="polar"))
figp.tight_layout()

# Visualise by area of bins
circular_hist(
    axp[0],
    relative_angles_of_impact,
    bins=8,
    density=True,
    offset=np.pi / 2,
    gaps=False,
)

circular_hist(
    axp[1],
    relative_angles_of_impact,
    bins=8,
    density=False,
    offset=np.pi / 2,
    gaps=False,
)


axp[0].title.set_text("Relative Impact Angles (Area Density)")
axp[1].title.set_text("Relative Impact Angles (Count)")

In [ ]:
df = pd.read_parquet(file_path)
severity_class_df = df[
    "severity_class_at_first_ego_collision_with_vehicles_stat_value"
]
severity_class = severity_class_df.to_numpy()

wc_severity_class_df = df[
    "severity_class_worst_case_at_first_ego_collision_with_vehicles_stat_value"
]
wc_severity_class = wc_severity_class_df.to_numpy()

mais3_driver_df = df[
    "mais3+_driver_at_first_ego_collision_with_vehicles_stat_value"
]
mais3_driver = mais3_driver_df.to_numpy()

mais3_passenger_df = df[
    "mais3+_passenger_at_first_ego_collision_with_vehicles_stat_value"
]
mais3_passenger = mais3_passenger_df.to_numpy()

mais3_backseat_df = df[
    "mais3+_backseat_at_first_ego_collision_with_vehicles_stat_value"
]
mais3_backseat = mais3_backseat_df.to_numpy()

wc_mais3_driver_df = df[
    "mais3+_worst_case_driver_at_first_ego_collision_with_vehicles_stat_value"
]
wc_mais3_driver = wc_mais3_driver_df.to_numpy()

wc_mais3_passenger_df = df[
    "mais3+_worst_case_passenger_at_first_ego_collision_with_vehicles_stat_value"
]
wc_mais3_passenger = wc_mais3_passenger_df.to_numpy()

wc_mais3_backseat_df = df[
    "mais3+_worst_case_backseat_at_first_ego_collision_with_vehicles_stat_value"
]
wc_mais3_backseat = wc_mais3_backseat_df.to_numpy()



fig, ax = plt.subplots(1, 4, figsize=(16, 4), constrained_layout=True)
#fig.tight_layout(pad=5)

plt.rcParams.update({'font.size': 14})

xticks = [0, 1, 2, 3]

for a in ax:
        a.set_ylim([0, 1])

ax[0].hist(severity_class, density=True, bins=np.arange(-1,4)+0.5, ec="k", alpha=0.5, range=(-0.5, 3.5))
ax[0].set_xticks(xticks)

ax[1].hist(wc_severity_class, density=True, bins=np.arange(-1,4)+0.5, ec="k", alpha=0.5, range=(-0.5, 3.5))
ax[1].set_xticks(xticks)

bins = np.arange(0, 100, 1)
ax[2].hist(mais3_backseat, bins=bins, density=True, ec="k", alpha=0.5)
ax[2].hist(mais3_passenger, bins=bins, density=True, ec="k", alpha=0.5)
ax[2].hist(mais3_driver, bins=bins, density=True, ec="k", alpha=0.5)

ax[3].hist(wc_mais3_backseat, bins=bins, density=True, ec="k", alpha=0.5)
ax[3].hist(wc_mais3_passenger, bins=bins, density=True, ec="k", alpha=0.5)
ax[3].hist(wc_mais3_driver, bins=bins, density=True, ec="k", alpha=0.5)




# ax[0].title.set_text("Severity Class")
ax[0].yaxis.set_label_text("Density")
ax[0].xaxis.set_label_text("Severity Class")
# ax[1].title.set_text("Worst Case Severity Class")
ax[1].yaxis.set_label_text("Density")
ax[1].xaxis.set_label_text("Severity Class")
ax[2].title.set_text("MAIS3+")
ax[2].yaxis.set_label_text("Density")
ax[2].legend(loc='upper right')
ax[2].xaxis.set_label_text("% chance of MAIS3+ injury")
ax[3].title.set_text("Worst Case MAIS3+")
ax[3].yaxis.set_label_text("Density")
ax[3].xaxis.set_label_text("% chance of MAIS3+ injury")
ax[3].legend(loc='upper right')

# fig.get_layout_engine().set(wspace=0)


fig, ax = plt.subplots(1, 1, figsize=(4, 4), constrained_layout=True)
#fig.tight_layout(pad=5)

plt.rcParams.update({'font.size': 14})

xticks = [0, 1, 2, 3]

ax.set_ylim([0, 1])

ax.hist(severity_class, density=True, bins=np.arange(-1,4)+0.5, ec="k", alpha=0.5, range=(-0.5, 3.5))
ax.set_xticks(xticks)


# ax[0].title.set_text("Severity Class")
ax.yaxis.set_label_text("Density")
ax.xaxis.set_label_text("Severity Class")







In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#########################################
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
folder = []
#seq
folder.append("../../data/nuplan/exp/exp/simulation/closed_loop_multiagent/choose_run")
#cross
folder.append("../../data/nuplan/exp/exp/simulation/closed_loop_multiagent/choose_run")
#merge
folder.append("../../data/nuplan/exp/exp/simulation/closed_loop_multiagent/choose_run")
#diverge
folder.append("../../data/nuplan/exp/exp/simulation/closed_loop_multiagent/choose_run")

plt.rcParams.update({'font.size': 10})

colors = ['tab:blue', 'tab:red', 'tab:green', 'tab:purple']
for i, f in enumerate(folder):
    file_path = f + "/metrics/" + "no_ego_at_fault_collisions.parquet"
    df = pd.read_parquet(file_path)

    dv = df[
        "first_ego_collision_energy_with_vehicles_stat_value"
    ]
    dv = dv.to_numpy() * 3.6

    impact_angle = df[
        "relative_impact_angle_at_first_ego_collision_with_vehicles_stat_value"
    ]
    theta = impact_angle.to_numpy()
    ax.set_theta_offset(np.pi / 2)
    ax.scatter(theta, dv, facecolors='none', edgecolors=colors[i], s=12)


ax.set_rmax(35)
ax.set_rticks(range(0, 40, 5))  # Less radial ticks
ax.set_rlabel_position(180-45)  # Move radial labels away from plotted lin
ax.grid(True)

lgnd = ax.legend(['Sequential', 'Cross', 'Merge', 'Diverge'], loc='upper left', bbox_to_anchor=(0.6, 0.3), title="Conflict Generator")
#change the marker size manually for both lines
for l in lgnd.legendHandles:
    l._sizes = [40]

# ax.set_title("Radial Plot of Impact Angles and Delta V", va='bottom')
ax.set_ylabel('delta V (Km/h)', rotation=0, size=10, loc='bottom')
plt.show()


In [ ]:
from matplotlib import cycler,colormaps

fig, ax = plt.subplots(1, 4, figsize=(16, 4), constrained_layout=True)
#fig.tight_layout(pad=5)

plt.rcParams.update({'font.size': 14})

xticks = [0, 1, 2, 3]

h = []

for i, f in enumerate(folder):
    file_path = f + "/metrics/" + "no_ego_at_fault_collisions.parquet"
    df = pd.read_parquet(file_path)

    impact_angle = df[
        "relative_impact_angle_at_first_ego_collision_with_vehicles_stat_value"
    ]
    severity_class_df = df[
        "severity_class_at_first_ego_collision_with_vehicles_stat_value"
    ]
    severity_class = severity_class_df.to_numpy()
    ax[i].set_ylim([0, 1])
    n, _, _ = ax[i].hist(severity_class, density=True, bins=np.arange(-1,4)+0.5, ec="k", alpha=0.5, range=(-0.5, 3.5))
    h.append(n)


plt.rcParams.update({'font.size': 16})
seq = h[0]
cross = h[1]
merge = h[2]
div = h[3]
sev = np.arange(4)

plt.figure(figsize=(7.5,4.5),layout='tight')
plt.rcParams['axes.prop_cycle']=cycler(color=colormaps['tab20b'].colors)
for i,(dens,lab) in enumerate(zip([cross,merge,div,seq],['Cross','Merge','Diverge','Sequential'])):
    plt.bar(sev+i/5,dens,width=0.2,label=lab)
plt.legend(title='Conflict Generator')
plt.ylim(0,1.04)
plt.xticks(sev+0.3,sev)
plt.xlabel('Severity Class')
plt.ylabel('Proportion')
plt.set_cmap('Blues')
plt.savefig('OCC_distributions.pdf',bbox_inches='tight',pad_inches=0.01)
plt.show()

In [ ]:
path = output_folder_occ + "/metrics/"
file_path_1 = path + "no_ego_at_fault_collisions.parquet"
df = pd.read_parquet(file_path_1)
df = df.loc[df["number_of_ego_collisions_with_vehicles_stat_value"] != 0]
df = df["scenario_name"]


scenario_list = df.tolist()
print("num : ", len(scenario_list))
for item in scenario_list:
    print(item)